In [ ]:
import numpy as np
from scipy.ndimage import convolve
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import time
import copy
import matplotlib.colors as mcolors
# Used for counting the number of living neighbors each cell has
FILTER = np.array([[1, 1, 1],
                   [1, 20, 1],
                   [1, 1, 1]], dtype=np.uint8)

In [ ]:
#PLOTTING

cmap = mcolors.ListedColormap([(1, 1, 1, 0), (0, 0, 0, 1)])  #settiamo i colori bianco=(1,1,1,0) e nero=(0,0,0,1)
bounds = [0, 0.5, 1]                                         #definisco un array di limiti che dividono l'intervallo in segmenti
norm = mcolors.BoundaryNorm(bounds, cmap.N)                  #la normalizzazione permette di mappare i valori della griglia a questi segmenti
def PlotGrid(grid, ax, fig,fine_spacing=1,t=0):
    """
    Visualize the current state of the Game of Life grid.

    Parameters
    ----------
    grid : 2D array
        Current state of the universe.
    ax : matplotlib.axes.Axes
        The axes object for plotting.
    fig : matplotlib.figure.Figure
        The figure object for plotting.
    """
    #clear_output(wait=True)
    ax.clear()     #pulizia dell'asse
    ax.imshow(grid, cmap=cmap,norm=norm, interpolation='nearest',extent=[0, grid.shape[1], 0, grid.shape[0]])   #utilizza le definizioni precedenti e extent definisce la dimensione dell'immagine

    # Fine gridlines
    ax.set_xticks(np.arange(0, grid.shape[1], fine_spacing), minor=False)    #imposto le linee della griglia
    ax.set_yticks(np.arange(0, grid.shape[0], fine_spacing), minor=False)    #separo con fine_spacing
    ax.grid(which="major", color="gray", linestyle='-', linewidth=1, alpha=0.7)

    # Remove axis ticks
    ax.tick_params(axis='both', which='both', length=0)     #rimozione dei tick degli assi per una visualizzazione più pulita
    ax.axes.xaxis.set_ticklabels([])
    ax.axes.yaxis.set_ticklabels([])
    ax.set_xlim(0, grid.shape[0])                           #imposto i limiti degli assi attraverso le dimensioni della griglia
    ax.set_ylim(0, grid.shape[1])
    #Display in Jupyter Notebook
    time.sleep(2)                                           #imposto pausa di 2 secondi per la visualizzazione su jupiter notebook dell'immagine
    display(fig)

In [ ]:
#PATTERNS AVAILABLE ON THE WEB

import requests      #libreria per effettuare richieste http
import re            #utile opoer ananlizzare file RLE, in generale per espressioni regolari

##Some Patterns availale through the website
patterns = [                                        #nomi di pattern presenti nel game of life di Conway
    "clock",
    "pulsar",
    "pentadecathlon",
    "blinker",
    "toad",
    "beacon",
    "glider",
    "lwss",  # Lightweight Spaceship
    "mwss",  # Middleweight Spaceship
    "hwss",  # Heavyweight Spaceship
    "caterpillar",
    "gosperglidergun",
    "simkinglidergun",
    "block",
    "beehive",
    "loaf",
    "boat",
    "tub",
    "r-pentomino",
    "diehard",
    "acorn",
    "switchengine"]

##Gets the pattern from web
def initial_state(length, pattern_name="random",config_param=0.5):          #crea una griglia per il game of life
    # Base URL for fetching patterns
    if pattern_name.lower()=="random":                                      #settato su random, sceglie a caso dalla lista
      p=config_param                                                        #solo se su 'random' prima, per probabilità di celle vive nelle celle (0.5 di default)
      grid = np.random.choice([0,1],p=[1-p,p], size=(length, length))
      return grid
    base_url = f"https://conwaylife.com/patterns/{pattern_name}.rle"        #viene costruito l'URL per scaricare il file RLE del pattern specificato

    try:                                                                    #GESTIONE DEGLI ERRORI  
        response = requests.get(base_url)
        response.raise_for_status()
        rle_data = response.text
    except requests.RequestException as e:
        raise ValueError(f"Error: '{pattern_name}': {e}")

    # Parse the RLE file to extract pattern dimensions and content                  
    pattern_lines = []
    x_size, y_size = None, None
    for line in rle_data.splitlines():
        if line.startswith("#") or line.strip() == "":
            continue
        if line.startswith("x ="):                                          #quando una riga parte con 'x=' si ricava le informazioni sulle dimensioni del pattern 
            match = re.search(r"x = (\d+), y = (\d+)", line)
            if match:
                x_size, y_size = int(match.group(1)), int(match.group(2))
        else:
            pattern_lines.append(line)

    if x_size is None or y_size is None:
        raise ValueError(f"Invalid RLE file format for pattern '{pattern_name}'.")

    pattern_grid = np.zeros((y_size, x_size), dtype=int)
    x, y = 0, 0
    for line in pattern_lines:                                              #costruisco la griglia con celle vive (1) e morte (0)
        count = ""                                                          #inoltre si analizza le righe
        for char in line:
            if char.isdigit():
                count += char
            elif char == "b":  # Dead cells
                count = int(count) if count else 1
                pattern_grid[y, x:x + count] = 0
                x += count
                count = ""
            elif char == "o":  # Alive cells
                count = int(count) if count else 1
                pattern_grid[y, x:x + count] = 1
                x += count
                count = ""
            elif char == "$":  # End of row
                count = int(count) if count else 1
                y += count
                x = 0
                count = ""
            elif char == "!":  # End of pattern
                break

    # Center the pattern in the grid
    grid = np.zeros((length, length), dtype=int)                            #c'entro il pattern in una griglia più grande
    start_x = (length - x_size) // 2
    start_y = (length - y_size) // 2
    grid[start_y:start_y + y_size, start_x:start_x + x_size] = pattern_grid

    return grid                                                              #infine la funzione restituisce la griglia così costruita

In [ ]:
#EVOLUTION OF THE SYSTEM THROUGH THE TIMESTEP

def evolve(length, T, config_type='random',config_param=0.5,plot=True):    
    """
    Run the Conway's Game of Life. Starting state is random.

    Parameters:
    ----------
    length : int  (Universe's dimension will be `length x length`)
    
    T : int       (Number of T to run simulation)
    ----------    
    """
    # Initialize the universe
    current = initial_state(length, config_type,config_param=config_param)    #inizializzo l'ambiente 2D array (griglia) nella configurazione iniziale
    next = np.empty_like(current)                                             #definisci un 'empty array' per poter metterci dentro lo stato attuale

    # Set up data storage
    state_history = [np.copy(current)]                                        #copio il sistema attuale in 'state_history', tiene traccia dell'evoluzione

    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(6, 6))

    # Show the initial board
    if plot:
      PlotGrid(current, ax, fig)

    # Run the simulation for the given number of generations
    for t in range(1,T+1):                                       #ciclo for per le simulazioni
    # Save the data of the current configuration

    # Advance the simulation
        next=NextStep(current, next)                            #array per il next step del gioco, quindi il prossimo stato del sistemo dopo aver evoluto a quello precedente
        current, next = next, current
        state_history.append(np.copy(current))
        if plot:
          PlotGrid(current, ax, fig,t=t)

    plt.close(fig)  # Close the figure when done
    #Output: data collected in the simulation to pass to data analysis function
    return state_history

In [ ]:
"APPLICAZIONE DELLE REGOLE DI GAME OF LIFE AL SISTEMA PER FARLO EVOLVERE ALLO STEP SUCCESSIVO"

In [ ]:
#FUNZIONE PER FARE EVOLVERE IL SISTEMA

def NextStep(current, next):
    """
    Calculate the next configuration of the system applying the rules of Game of Life.

    Parameters
    ----------
    current : 2D array     (Current state of universe)

    next : 2D array  (empty to host the next state of the universe, so it must have the same size of the 'current' array)
    ---------- 
    """
    next[:] = 0
    #print(next.shape)
    count = convolve(current, FILTER, mode="wrap")
    next[(count == 3) | (count == 22) | (count == 23)] = 1
    return next

In [ ]:
#DISTANZA QUADRATICA MEDIA TRA CELLULA VIVA E CENTRO DI MASSA (se ci sono celle vive)

def square_distance(grid):
    coordinates = np.argwhere(grid == 1)
    if len(coordinates) > 0:
        center_of_mass = np.mean(coordinates, axis=0)
        distances_squared = np.sum((coordinates - center_of_mass)**2, axis=1)
        return np.mean(distances_squared)
    return 0

In [ ]:
#FUNZIONE PER TROVARE PERIODICITA' NELLE CONFIGURAZIONI

def find_periodicity(states):
    seen_states = {}                           #per tenere traccia sia degli stati precedenti che dei loro corrispettivi time-step

    for t, state in enumerate(states):
        # Convert the matrix to a hashable representation
        state_hash = state.tobytes()           #converte lo stato corrente del sistema in bytes, cosi che sia più facile da controllare o utlizzabili come chiavi dei dizionari 

        if state_hash in seen_states:          #se una configurazione è già stata riscontrata, ci indica in che istante si è ripetuto
            previous_index = seen_states[state_hash]
            return t, previous_index

        seen_states[state_hash] = t

    # No periodicity detected
    return len(states)

In [ ]:
#ANALISI DELLA SIMULAZIONE

def analyze_base_game_of_life(state_history):
    """
    Analyze the evolution of the Game of Life system, including live cell count, unique states, and oscillators.

    Parameters
    ----------
    live_cells : list
        Number of live cells in each generation.
    unique_states : set
        Unique states encountered during the evolution.
    state_history : list
        Complete history of states (array).
    spread_distances : list
        Mean square distance of individuals over time.
    """
    live_cells=[np.sum(current) for current in state_history]                   #array che contiene il numero di celle vive step per step
    spread_distances=[square_distance(current) for current in state_history]    #calcolo della distanza quadratica media dal centro di massa del sistema
    generations = np.arange(len(live_cells))                                    #array di dimensione pari al numero totale di generazioni (perciò step)
    total_cells = state_history[0].size                                         #calcolo di numero totale di celle al primo istante

    # Create a figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(10, 7))                             #griglia con due sottotrame
    fig.suptitle("Game of Life Analysis", fontsize=16)

    # Subplot 1: Live cell evolution
    ax1 = axes[0, 0]
    ax1.plot(generations, live_cells, label="Alive cells", color='blue')
    ax1.set_title("Time Evolution of Alive Cells")
    ax1.set_xlabel("Generation")
    ax1.set_ylabel("Alive cells")
    ax1.legend()
    ax1.grid()

    # Subplot 2: Mean square distance (R^2)
    if spread_distances is not None:
        ax2 = axes[0, 1]
        ax2.plot(generations, spread_distances, label='R^2(t)', color='orange')
        ax2.set_title("Mean Square Distance (R^2)")
        ax2.set_xlabel("Generation")
        ax2.set_ylabel("R^2")
        ax2.legend()
        ax2.grid()
    else:
        axes[0, 1].axis('off')  # Hide this subplot if no data is available

    # Subplot 3: Initial and final states
    ax3 = axes[1, 0]
    ax3.imshow(state_history[0], cmap='binary')
    ax3.axis('off')
    ax3.set_title("First State")

    ax4 = axes[1, 1]
    ax4.imshow(state_history[-1], cmap='binary')
    ax4.axis('off')
    ax4.set_title("Last State")

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit the title

    # Text output
    #avg_occupancy = np.mean(live_cells) / total_cells * 100                         #percentuale di celle vive rispetto al totale
    occupancy_per_step = (live_cells / total_cells) * 100
    avg_occupancy = np.mean(occupancy_per_step)
    print(f"Average Occupancy Percentage: {avg_occupancy:.2f}%")

    # Oscillators
    Period=find_periodicity(state_history)
    print("With ", len(state_history)," steps we have a periodicity of ",Period)
